## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Asgard

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql 
   CREATE TABLE db1.cameras_st_asgard
(
    `report_date` Date,
    `camera_uuid` String,
    `camera_dvr_depth` Int16,
    `camera_is_permanent_stream` Int16,
    `camera_with_intercom` Int16,
    `partner_uuid` String,
    `camera_fw_version` String,
    `camera_model` String,
    `camera_streamer` String
)
ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/cameras_st_asgard/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE TABLE db1.cameras_st_asgard_ch
(
    `report_date` Date,
    `camera_uuid` String,
    `camera_dvr_depth` Int16,
    `camera_is_permanent_stream` Int16,
    `camera_with_intercom` Int16,
    `partner_uuid` String,
    `camera_fw_version` String,
    `camera_model` String,
    `camera_streamer` String
)
    ENGINE = MergeTree()
    ORDER BY camera_uuid
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.cameras_st_asgard_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.cameras_st_asgard_ch AS
    SELECT
        *
    FROM db1.cameras_st_asgard
    WHERE partner_uuid not like '%main:tokens:%'
    """

ch.query_run(query_text)

___
## Tools
___
### query


In [2]:
query_text = """--sql
    SELECT
        *
    FROM db1.cameras_st_asgard_ch
    ORDER BY report_date desc
    limit 100
    """

ch.query_run(query_text)


report_date,camera_uuid,camera_dvr_depth,camera_is_permanent_stream,camera_with_intercom,partner_uuid,camera_fw_version,camera_model,camera_streamer
date,str,i64,i64,i64,str,str,str,str
2025-07-30,"""00143259-456e-4f81-b9c3-439a57…",0,0,1,"""7775bc0f-24e5-4afc-9054-e194c8…","""""","""""","""cameras.sputnik.systems"""
2025-07-30,"""002768ab-1b49-4796-b287-d087dd…",0,0,0,"""0e7236ac-b8bd-4ab0-8634-d165ad…","""""","""""","""cameras.sputnik.systems"""
2025-07-30,"""0029385e-a17c-40d1-8528-2678e0…",0,0,1,"""ba308431-fd0c-4921-a605-9e21c5…","""""","""""",""""""
2025-07-30,"""0001abf7-f735-4962-b5a0-f5ff98…",0,0,1,"""ba308431-fd0c-4921-a605-9e21c5…","""""","""""",""""""
2025-07-30,"""0013d243-1413-4b51-8d96-9f381d…",0,0,1,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""""","""""",""""""
…,…,…,…,…,…,…,…,…
2025-07-30,"""002f62e6-7cf4-4db1-abf1-1caa57…",3,1,1,"""0e7236ac-b8bd-4ab0-8634-d165ad…","""4.12""","""MC200J2""","""msk-v04.sputnik.systems"""
2025-07-30,"""003afa04-8a5f-4e36-beb9-402de9…",0,0,0,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""5.03""","""MCJ12""","""msk-v05.sputnik.systems"""
2025-07-30,"""00484ba6-3a31-43e8-b2ff-e682cc…",0,0,1,"""f1f4d289-beb1-44ad-954b-42a8f0…","""""","""""","""msk-v06.sputnik.systems"""


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.cameras_st_asgard_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.cameras_st_asgard_mv
    """

ch.query_run(query_text)

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.cameras_st_asgard_ch
    """

ch.query_run(query_text)

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.cameras_st_asgard_mv
"""

ch.query_run(query_text)